# Seq2Seq作业

本次作业的目的是使用Seq2Seq模型进行法语-英语机器翻译任务。请先使用`pip install sacrebleu`安装sacrebleu库用于计算BLEU评测指标。

本次作业侧重于对代码的理解和对实验结果的分析，模型表现不会作为评分依据。

**截止时间：6月5日**

## 1. 读取数据

本次作业使用的数据位于`fr-en.txt`文件中，每一行是一组数据，形式为“法语句子\t英语句子”。其中的法语句子和英语句子均已经过预处理，可以直接按照空格切分为单词。

In [1]:
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from sacrebleu.metrics import BLEU
from tqdm import tqdm

d:\Anaconda3\envs\pytorchcuda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fr_sents = []
en_sents = []
with open("fr-en.txt") as f:
    for line in f:
        fr, en = line.strip().split("\t")
        fr = fr.split()
        en = en.split()
        fr_sents.append(fr)
        en_sents.append(en)

将数据打乱后按照8:1:1切分为训练集、验证集和测试集。

In [3]:
idx = list(range(len(fr_sents)))
random.shuffle(idx)
_fr_sents = [fr_sents[_] for _ in idx]
_en_sents = [en_sents[_] for _ in idx]

N = len(fr_sents)
N_train = int(N * 0.8)
N_valid = int(N * 0.1)
N_test = N - N_train - N_valid

train_fr = _fr_sents[:N_train]
train_en = _en_sents[:N_train]
valid_fr = _fr_sents[N_train:N_train+N_valid]
valid_en = _en_sents[N_train:N_train+N_valid]
test_fr = _fr_sents[N_train+N_valid:]
test_en = _en_sents[N_train+N_valid:]

print("训练集 验证集 测试集大小分别为", N_train, N_valid, N_test)

训练集 验证集 测试集大小分别为 16000 2000 2000


定义词表类Vocab，用于记录两种语言中出现的单词及其编号。

In [4]:
class Vocab():
    def __init__(self):
        self.word2idx = {}
        self.word2cnt = {}
        self.idx2word = []
        self.add_word("[BOS]")
        self.add_word("[EOS]")
        self.add_word("[UNK]")
    
    def add_word(self, word):
        """
        将单词word加入到词表中
        """
        if word not in self.word2idx:
            self.word2cnt[word] = 0
            self.word2idx[word] = len(self.idx2word)
            self.idx2word.append(word)
        self.word2cnt[word] += 1
    
    def add_sent(self, sent):
        """
        将句子sent中的每一个单词加入到词表中
        sent是由单词构成的list
        """
        for word in sent:
            self.add_word(word)
    
    def index(self, word):
        """
        若word在词表中则返回其下标，否则返回[UNK]对应序号
        """
        return self.word2idx.get(word, self.word2idx["[UNK]"])
    
    def encode(self, sent):
        """
        在句子sent的首尾分别添加BOS和EOS之后编码为整数序列
        """
        encoded = [self.word2idx["[BOS]"]] + [self.index(word) for word in sent] + [self.word2idx["[EOS]"]]
        return encoded
    
    def decode(self, encoded, strip_bos_and_eos=False):
        """
        将整数序列解码为单词序列
        """
        return [self.idx2word[_] for _ in encoded if not strip_bos_and_eos or self.idx2word[_] not in ["[BOS]", "[EOS]"]]
    
    def __len__(self):
        """
        返回词表大小
        """
        return len(self.idx2word)

对于两种语言分别构建词表。

In [5]:
fr_vocab = Vocab()
en_vocab = Vocab()

for fr, en in zip(train_fr, train_en):
    fr_vocab.add_sent(fr)
    en_vocab.add_sent(en)

print("法语词表大小为", len(en_vocab))
print("英语词表大小为", len(fr_vocab))

法语词表大小为 3130
英语词表大小为 5453


## 2. Seq2Seq模型

### 1) Attention
机器翻译中，目标语言中的每个单词往往都对应于源语言中的一个或几个单词，在翻译时，如果模型能够学习到这种对应关系，则可以一定程度上缓解RNN中的长程信息损失问题，对翻译效果带来很大提升。

Attention机制正是基于这样的思想，可以在解码过程中关注到编码器端不同时间片的信息。在解码过程中的每个时间片，attention会利用当前解码端的隐状态，对编码端的每个时间步计算一个相关性分数。这个相关性分数使用Softmax进行归一化后，作为权重对编码端的隐状态序列加权求和，即得到解码端当前时间片的注意力向量。这个注意力向量中包含了源语言中和当前时间片最相关的信息，可以用于当前时间片的输出预测及隐状态更新。

记当前解码端隐状态为$\mathbf{h_t}$, 编码器端的隐状态为$\mathbf{s_j}, j=0 \ldots L-1$, 一些常见的相关性分数计算方法为：
- $\alpha_{t,j}=\mathbf{h_t}^T \mathbf{s_j}$
- $\alpha_{t,j}=\mathbf{h_t}^T W \mathbf{s_j}$
- $\alpha_{t,j}=\mathbf{v}^T \mathrm{tanh}(W_1 \mathbf{h_t} + W_2 \mathbf{s_j})$

其中$W, W_1, W_2, \mathbf{v}$都为神经网络的可学习参数。解码端该时间片的注意力向量为 $\mathbf{c_t} = \sum_{j=0}^{L-1} \alpha_{t,j}\mathbf{s_j}$。

### 2) 本次作业中使用的Seq2Seq模型（也可直接阅读代码）
本次作业使用的Seq2Seq模型如下（其中N为batch size，H为hidden size）：
- 注意力模块Attention
    - 输入包括解码器端的隐状态`decoder_hidden_state`（大小为N \* H）和编码器端的隐状态序列`encoder_hidden_states`（大小为N \* L \* H，其中L为源语言长度）
    - 对于batch中的第i个样例，Attention的输出向量是对编码器端隐状态序列的加权求和
$$ \mathrm{attn\_output}_i = \sum_{t=0}^{L-1} \alpha_{i,t} ~\mathrm{Enc_{i, t}} $$
    其中权重采取1)中的第二种计算方法，即$\alpha_{i,t} = \mathrm{Dec}_{i}^T ~ W ~ \mathrm{Enc}_{i,t}$，其中$W$是个可学习的线性变换。
    - 输出的`attn_output`大小为N * H。
- 编码器EncoderRNN
    - 输入包括源语言单词下标序列`input`（大小为N）以及RNN初始隐状态`hidden`（大小为N * H，初始化为0向量）；
    - 本次作业使用[nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)对词向量进行学习，词向量的维数设为`embedding_dim`;
    - 本次作业使用pytorch提供的[nn.GRUCell](https://pytorch.org/docs/stable/generated/torch.nn.GRUCell.html);
    - 在每个时间片，使用Embedding将单词的下标映射为对应的词向量（大小为N \* embedding_dim），作为RNN的输入向量，连同上一个时间片的`hidden`一起输入到GRUCell中，得到当前时间片的`hidden`；
    - 由于在编码器端不做预测，因此不需要得到output；
    - 输出当前时间片的隐状态`hidden`，大小为N \* H。
- 解码器DecoderRNN
    - 输入包括目标语言单词下标序列`input`（大小为N）、RNN初始隐状态`hidden`（大小为N \* H，初始化为编码器的最终隐状态）以及编码器端隐状态序列`encoder_hiddens`（大小为N \* L \* H）；
    - 类似于EncoderRNN，使用nn.Embedding和nn.GRUCell，并且使用了之前定义的注意力模块Attention；
    - 在每个时间片，依次执行
        - 将`input`通过Embedding映射为词向量；
        - 将词向量与前一个时间片的隐状态concat起来，经过一个线性变换`h2q`后，对编码器隐状态序列做attention，得到`attn_output`；
        - 将`input`与`hidden`输入到GRUCell中更新隐状态`hidden`；
        - 将词向量与attention的结果concat起来作为GRUCell的输入向量，与`hidden`一起输入到GRUCell中，得到当前时间片的隐状态`hidden`；
        - 将`hidden`与attention的结果concat起来，经过线性变换`h2o`和LogSoftmax得到输出`output`；
    - 输出包括当前时间片的隐状态`hidden`（大小为N \* H）和输出`output`（大小为N \* V，其中V为目标语言的词表大小）
- Seq2Seq类包含了一个编码器模块和一个解码器模块，训练时解码器端使用teacher forcing（使用标准答案的单词编号作为输入，而非模型的预测结果），预测时解码器端使用贪心的解码策略（每个时间片预测概率最大的单词作为下一个时间片的输入）。

为简便起见，本次实现中Seq2Seq中的所有数据的batch size都可看作1。

In [19]:
MAX_LEN = 10    # 最大解码长度

class Attention(nn.Module):
    def __init__(self, hidden_size):
        """
        假定编码器和解码器的hidden size相同。
        """
        super(Attention, self).__init__()
        self.lin = nn.Linear(hidden_size, hidden_size, bias=False)
    
    def forward(self, encoder_hidden_states, decoder_hidden_state):
        """
        encoder_hidden_states: N * L * H
        decoder_hidden_state: N * H
        L为源语言长度，H为hidden size

        输出attn_output（大小为N * H）
        """
        dh = self.lin(decoder_hidden_state).unsqueeze(-1)     # N * H * 1
        attn_scores = torch.bmm(encoder_hidden_states, dh)    # N * L * 1 注意力（相关性）分数
        weights = F.softmax(attn_scores, dim=1)               # 在L维度上归一化得到权重
        outputs = (weights * encoder_hidden_states).sum(1)    # N * H    在L维度上加权求和
        return outputs

class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(EncoderRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRUCell(embedding_dim, hidden_size)
    
    def forward(self, input, hidden):
        """
        input: N
        hidden: N * H
        
        输出更新后的隐状态hidden（大小为N * H）
        """
        embedding = self.embed(input)
        hidden = self.gru(embedding, hidden)
        return hidden

class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(DecoderRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRUCell(embedding_dim + hidden_size, hidden_size)
        self.attn = Attention(hidden_size)
        self.h2q = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self, input, hidden, encoder_hiddens):
        """
        input: N
        hidden: N * H
        encoder_hiddens: N * L * H
        
        输出对于下一个时间片的预测output（大小为N * V）更新后的隐状态hidden（大小为N * H）
        """
        embedding = self.embed(input)
        attn_query = self.h2q(torch.cat((embedding, hidden), dim=-1))
        attn_output = self.attn(encoder_hiddens, attn_query)
        input_combined = torch.cat((embedding, attn_output), dim=-1)
        hidden = self.gru(input_combined, hidden)
        output = self.h2o(attn_output)
        output = self.softmax(output)
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, embedding_dim, hidden_size, teaching_forcing_ratio):
        super(Seq2Seq, self).__init__()
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.hidden_size = hidden_size
        self.encoder = EncoderRNN(len(src_vocab), embedding_dim, hidden_size)
        self.decoder = DecoderRNN(len(tgt_vocab), embedding_dim, hidden_size)
        self.teaching_forcing_ratio = teaching_forcing_ratio
        
        self.register_buffer("_hidden", torch.zeros(1, hidden_size))
        self.register_buffer("_tgt_bos", torch.full((1, ), tgt_vocab.index("[BOS]"), dtype=torch.long))
    
    def init_hidden(self):
        """
        初始化编码器端隐状态为全0向量（大小为1 * H）
        """
        return torch.zeros_like(self._hidden)
    
    def init_tgt_bos(self):
        """
        预测时，初始化解码器端输入为[BOS]（大小为1）
        """
        return torch.full_like(self._tgt_bos, self.tgt_vocab.index("[BOS]"))
    
    def forward_encoder(self, src):
        """
        src: N * L
        编码器前向传播，输出最终隐状态hidden (N * H)和隐状态序列encoder_hiddens (N * L * H)
        """
        _, Ls = src.size()
        hidden = self.init_hidden()
        encoder_hiddens = []
        # 编码器端每个时间片，取出输入单词的下标，与上一个时间片的隐状态一起送入encoder，得到更新后的隐状态，存入enocder_hiddens
        for i in range(Ls):
            input = src[:, i]
            print(input)
            hidden = self.encoder(input, hidden)
            encoder_hiddens.append(hidden)
        encoder_hiddens = torch.stack(encoder_hiddens, dim=1)
        return hidden, encoder_hiddens
    
    def forward_decoder(self, tgt, hidden, encoder_hiddens):
        """
        tgt: N
        hidden: N * H
        encoder_hiddens: N * L * H
        
        解码器前向传播，用于训练，使用teacher forcing，输出预测结果outputs，大小为N * L * V，其中V为目标语言词表大小
        """
        _, Lt = tgt.size()
        outputs = []
        for i in range(Lt):
            ratio = random.random()
            if ratio >= self.teaching_forcing_ratio and len(outputs) > 0:
                input = torch.tensor([torch.argmax(outputs[-1][:])]).to(device)  # 非teacher forcing，使用上一个预测出的单词作为输入
            else:
                input = tgt[:, i]    # teacher forcing, 使用标准答案的单词作为输入，而非模型预测值
            #print(len(outputs), input.shape, input, tgt[:, i])
            output, hidden = self.decoder(input, hidden, encoder_hiddens)
            outputs.append(output)
        outputs = torch.stack(outputs, dim=1)
        return outputs
        
    
    def forward(self, src, tgt):
        """
            src: 1 * Ls
            tgt: 1 * Lt
            
            训练时的前向传播
        """
        print(src)
        hidden, encoder_hiddens = self.forward_encoder(src)
        outputs = self.forward_decoder(tgt, hidden, encoder_hiddens)
        return outputs
    
    def predict(self, src):
        """
            src: 1 * Ls
            
            用于预测，解码器端初始输入为[BOS]，之后每个位置的输入为上个时间片预测概率最大的单词
            当解码长度超过MAX_LEN或预测出了[EOS]时解码终止
            输出预测的单词编号序列，大小为1 * L，L为预测长度
        """
        hidden, encoder_hiddens = self.forward_encoder(src)
        input = self.init_tgt_bos()
        preds = [input]
        while len(preds) < len(src[0]) and len(preds) < 10:
            output, hidden = self.decoder(input, hidden, encoder_hiddens)
            input = output.argmax(-1)
            preds.append(input)
            if input == self.tgt_vocab.index("[EOS]"):
                break
        preds = torch.stack(preds, dim=-1)
        return preds

下面这段代码创建了对应的DataLoader，从中加载得到的每一个batch由两个list构成，每个list包含了batch size个tensor，其中
- 第一个list中的tensor对应于源语言的单词编号序列；
- 第一个list中的tensor对应于目标语言的单词编号序列；
- 每个tensor大小为即为序列的长度L，其中第一个元素对应于\[BOS\]，最后一个元素对应于\[EOS\]。

In [7]:
def collate(data_list):
    src = [torch.tensor(_[0]) for _ in data_list]
    tgt = [torch.tensor(_[1]) for _ in data_list]
    return src, tgt

batch_size = 16
trainloader = torch.utils.data.DataLoader([
    (fr_vocab.encode(fr), en_vocab.encode(en)) for fr, en in zip(train_fr, train_en)
], batch_size=batch_size, shuffle=True, collate_fn=collate)
validloader = torch.utils.data.DataLoader([
    (fr_vocab.encode(fr), en_vocab.encode(en)) for fr, en in zip(valid_fr, valid_en)
], batch_size=batch_size, shuffle=False, collate_fn=collate)
testloader = torch.utils.data.DataLoader([
    (fr_vocab.encode(fr), en_vocab.encode(en)) for fr, en in zip(test_fr, test_en)
], batch_size=batch_size, shuffle=False, collate_fn=collate)

In [ ]:
for x, y in trainloader:
    print(x)
    print(y)
    break

训练和预测的代码如下。

In [9]:
device = torch.device("cpu")  # 训练过程使用CPU耗时约15分钟，使用RTX2080Ti耗时约6分钟

In [11]:
def train_loop(model, optimizer, criterion, loader):
    model.train()
    epoch_loss = 0.0
    for src, tgt in tqdm(loader):
        B = len(src)
        loss = 0.0
        for _ in range(B):
            _src = src[_].unsqueeze(0).to(device)     # 1 * L
            _tgt = tgt[_].unsqueeze(0).to(device)     # 1 * L
            outputs = model(_src, _tgt)     # 1 * L * V
            
            # decoder端，每个位置的输出预测的是下一个位置的单词，因此需要错一位计算loss
            loss += criterion(outputs[:,:-1,:].squeeze(0), _tgt[:,1:].squeeze(0))
        
        loss /= B
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)     # 裁剪梯度，将梯度范数裁剪为1，使训练更稳定
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(loader)
    return epoch_loss


def test_loop(model, loader, tgt_vocab):
    model.eval()
    bleu = BLEU(force=True)
    hypotheses, references = [], []
    for src, tgt in tqdm(loader):
        B = len(src)
        for _ in range(B):
            _src = src[_].unsqueeze(0).to(device)     # 1 * L
            with torch.no_grad():
                outputs = model.predict(_src)         # 1 * L
            
            # 保留预测结果，使用词表vocab解码成文本，并删去BOS与EOS
            ref = " ".join(tgt_vocab.decode(tgt[_].tolist(), strip_bos_and_eos=True))
            hypo = " ".join(tgt_vocab.decode(outputs[0].cpu().tolist(), strip_bos_and_eos=True))
            references.append(ref)    # 标准答案
            hypotheses.append(hypo)   # 预测结果
    
    score = bleu.corpus_score(hypotheses, [references]).score      # 计算BLEU分数
    return hypotheses, references, score

In [20]:
torch.manual_seed(1)
for ratio in range(3):
    ratio = ratio*0.5
    model = Seq2Seq(fr_vocab, en_vocab, embedding_dim=256, hidden_size=256, teaching_forcing_ratio=ratio)
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1)
    criterion = nn.NLLLoss()
    print("Current teaching forcing ratio is {}".format(ratio))
    best_score = 0.0
    for _ in range(2):
        loss = train_loop(model, optimizer, criterion, trainloader)
        hypotheses, references, bleu_score = test_loop(
            model, validloader, en_vocab)
        # 保存验证集上bleu最高的checkpoint
        if bleu_score > best_score:
            torch.save(model.state_dict(), "model_best.pt")
            best_score = bleu_score
        print(f"Epoch {_}: loss = {loss}, valid bleu = {bleu_score}")
        print(references[0])
        print(hypotheses[0])
    model.load_state_dict(torch.load("model_best.pt"))
    hypotheses, references, bleu_score = test_loop(model, testloader, en_vocab)
    print(f"Test bleu = {bleu_score}")
    print(references[0])
    print(hypotheses[0])

Current teaching forcing ratio is 0.0


  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([[  0,  24, 865, 652, 653, 654,   9,   1]])
tensor([[  0,  24, 865, 652, 653, 654,   9,   1]])
tensor([0])
tensor([[-1.5256e+00, -7.5023e-01, -6.5398e-01, -1.6095e+00, -1.0017e-01,
         -6.0919e-01, -9.7977e-01, -1.6091e+00, -7.1214e-01,  3.0372e-01,
         -7.7731e-01, -2.5146e-01, -2.2227e-01,  1.6871e+00,  2.2843e-01,
          4.6764e-01, -6.9697e-01, -1.1608e+00,  6.9954e-01,  1.9908e-01,
          8.6569e-01,  2.4440e-01, -6.6291e-01,  8.0731e-01,  1.1017e+00,
         -1.7594e-01, -2.2456e+00, -1.4465e+00,  6.1155e-02, -6.1774e-01,
         -7.9807e-01, -1.3162e-01,  1.8793e+00, -7.2132e-02,  1.5777e-01,
         -7.7345e-01,  1.9906e-01,  4.5703e-02,  1.5296e-01, -4.7568e-01,
         -1.1102e-01,  2.9274e-01, -1.5785e-01, -2.8787e-02,  2.3571e+00,
         -1.0373e+00,  1.5748e+00, -6.2985e-01, -9.2739e-01,  5.4514e-01,
          6.6280e-02, -4.3704e-01,  7.6260e-01,  4.4151e-01,  1.1651e+00,
          2.0154e+00,  1.3741e-01,  9.3864e-01, -1.8600e-01, -6.4464e-01

  0%|          | 0/1000 [00:04<?, ?it/s]


KeyboardInterrupt: 

加载验证集上bleu最高的模型，在测试集上进行评测。

In [1]:
# 已经在上面输出了
model.load_state_dict(torch.load("model_best.pt"))
hypotheses, references, bleu_score = test_loop(model, testloader, en_vocab)
print(f"Test bleu = {bleu_score}")
print(references[0])
print(hypotheses[0])

NameError: name 'model' is not defined

**作业要求：** 理解关于Seq2Seq模型部分的代码，**分别**进行以下两处修改并进行实验：
1. 上面代码中，解码器共同利用hidden与attn_output得到输出，**请你修改为只使用attention向量预测输出的方案，并进行实验**；
2. 上面代码实现了训练时使用teacher forcing的方案，**请你在模型中加入一个可以手动调整的参数teacher_forcing_ratio，并修改forward_decoder方法，使得在训练时以概率p=teacher_forcing_ratio使用teacher forcing，以1-p的概率不使用teacher forcing（即使用模型预测的输出作为下一个时间片的输入），并在这个参数为0, 0.5, 1时分别进行实验，并比较实验结果**。

*注意训练时即使不使用teacher forcing，也应保持长度与真实答案一致，以便于计算loss。*

## 3. 附加题

**可选** 实现beam search，并在测试集上计算BLEU分数。

In [ ]:
@torch.no_grad()
def beam_search(model, src, beam_size=5):
    # TODO
    raise NotImplementedError


model.eval()
bleu = BLEU(force=True)
hypotheses, references = [], []
for src, tgt in tqdm(testloader):
    B = len(src)
    for _ in range(B):
        # TODO
        raise NotImplementedError

score = bleu.corpus_score(hypotheses, [references]).score      # 计算BLEU分数
print(f"Beam search (beam size = 5): BLEU = {score}")